## Function Definitions

In [109]:
import json
import uuid
from functools import partial


def add(args: str):
    args = json.loads(args)
    return str(float(args["a"]) + float(args["b"]))


def sub(args: str):
    args = json.loads(args)
    return str(float(args["a"]) - float(args["b"]))


def mult(args: str):
    args = json.loads(args)
    return str(float(args["a"]) * float(args["b"]))


def div(args: str):
    args = json.loads(args)
    return str(float(args["a"]) / float(args["b"]))


def get_oai_response(model, functions, msgs, api_key, base_url):
  import openai
  openai.api_key = api_key ## Add your API key here
  openai.base_url = base_url
  print(f"Pointing to URL: {base_url}")
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      # functions=functions,
      # function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e)


def insert_tool_response(res, msgs):
    # for tool_call in res.message.tool_calls:
    #     print(f"Tool Call: {tool_call.id}, {tool_call.function}")
    assistant_message = res.message
    tool_calls = []
    for tool_call in assistant_message.tool_calls:
        tool_calls.append( {
                            "id": tool_call.id,
                            "function": {"name": tool_call.function.name,
                                        "arguments": tool_call.function.arguments},
                            "type": "function",
                        })
    msgs.append({"role": "assistant",  "tool_calls": tool_calls})
    
    for i, tool_call in enumerate(assistant_message.tool_calls):
        if tool_call.function.name == "getCurrentWeather":
            print()
            l = len((json.loads(assistant_message.tool_calls[i].function.arguments))["location"])
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"temprature is {(i+1) * 50 + l } degree"})
        elif tool_call.function.name == "calculate_distance":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Distance is {(i+1) * 50} miles."})
        elif tool_call.function.name == "generate_password":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Password generated: {uuid.uuid4().hex[:8]}"})
        elif tool_call.function.name == "orderUmbrella":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Order placed. the price is {(i+1) * 10} dollars."})
        elif tool_call.function.name == "list_files":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"File list:\nreport.docx\ntask.txt\nnotes.txt"})
        elif tool_call.function.name == "get_file_size":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"the size is {(i+1) * 100} bytes."})
        elif tool_call.function.name == "addition":
            msgs.append({
                "role": "tool",
                "name": "addition",
                "content": add(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "subtraction":
            msgs.append({
                "role": "tool",
                "name": "subtraction",
                "content": sub(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "multiplication":
            msgs.append({
                "role": "tool",
                "name": "multiplication",
                "content": mult(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "division":
            msgs.append({
                "role": "tool",
                "name": "division",
                "content": div(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        print(f"Observation: {msgs[-1]}")
    
    return msgs

def run_completion(chat_method, user_query, msgs=[]):
    system_prompt = "You are a helpful assistant."
    functions = [
        # {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}},
        {
            "type": "function",
            "function": {
                "name": "dummy",
                "description": "just to say hi",
                "parameters": None,
            }
        },
        {
            "type": "function",
            "function": {
                "name": "list_files",
                "description": "List all files in a directory",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "directory": {
                            "type": "string",
                            "description": "the directory to list files from"
                        }
                    },
                    "required": [
                        "directory"
                    ]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Create a 3D model of an object with specified dimensions",
                "name": "create_3d_model",
                "parameters": {
                    "properties": {
                    "object_name": {
                        "description": "Name of the object to be modeled",
                        "type": "string"
                    },
                    "dimensions": {
                        "description": "Dimensions of the 3D object (length, width, height)",
                        "type": "object",
                        "properties": {
                        "length": {
                            "type": "number"
                        },
                        "width": {
                            "type": "number"
                        },
                        "height": {
                            "type": "number"
                        }
                        },
                        "required": [
                        "length",
                        "width",
                        "height"
                        ]
                    }
                    },
                    "required": [
                    "object_name",
                    "dimensions"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Get the latest insurance premium from a list of premiums.",
                "name": "latest_insurance_premium",
                "parameters": {
                    "properties": {
                        "premiums": {
                            "description": "List of insurance premiums",
                            "type": "array",
                            "items": {
                            "type": "number"
                            }
                        }
                    },
                    "required": [
                        "premiums"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
            "description": "Calculate insurance premium based on age and coverage",
            "name": "calculate_insurance_premium",
            "parameters": {
                "properties": {
                "age": {
                    "description": "Age of the person applying for insurance",
                    "type": "integer"
                },
                "coverage_type": {
                    "description": "Type of insurance coverage",
                    "type": "string",
                    "enum": [
                    "basic",
                    "standard",
                    "premium"
                    ]
                }
                },
                "required": [
                "age",
                "coverage_type"
                ],
                "type": "object"
            }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_file_size",
                "description": "Get the size of a file in bytes",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "the name of the file to get its size"
                        }
                    },
                    "required": [
                        "filename"
                    ]
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'addition',
                'description': "Adds two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to add',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to add',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'subtraction',
                'description': "Subtracts two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to be subtracted from',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Number to subtract',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'multiplication',
                'description': "Multiply two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to multiply',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to multiply',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'division',
                'description': "Divide two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to use as the dividend',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to use as the divisor',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
      {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}
       ]
    # functions = [{"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}]

    if not msgs or len(msgs) == 0:
        msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]
    else:
        msgs.append({"role": "user", "content": user_query})

    res = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
    res_next = res
    if res_next.message.content and len(res_next.message.content) > 0:
        print("\n[AI response]:\n", res_next.message.content)
    else:
        print("\n[AI calling functions]:")
        for tool_call in res_next.message.tool_calls:
            print(f"Tool Call: {tool_call.function}")
    l = 0
    while res_next.message.tool_calls and len(res_next.message.tool_calls) > 0:
        msgs = insert_tool_response(res_next, msgs)
        print(msgs)
        res_next = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
        # for m in msgs:
        #     print(m)
        print(f"Loop {l}")
        if res_next.message.content and len(res_next.message.content) > 0:
            print("\n[AI response]:\n", res_next.message.content)
        else:
            print("\n[AI calling functions]:")
            for tool_call in res_next.message.tool_calls:
                print(f"Tool Call: {tool_call.function}")
        l += 1
    

## Multi + Parallel Function Call

In [110]:
import openai
local_api_key = "sk-"
local_base_url = "http://localhost:1234/v1/"

In [3]:

get_mistral_rubra_response = partial(get_oai_response, api_key=local_api_key, base_url=local_base_url)

user_query = "What is the distance between San Francisco and Cupertino by driving and by air from both directions?"
# user_query = "What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"
# user_query = "what's the distance between SF and NYC? Use the result value to multiply by 8, and then divide by 2, and then minus 30"
msgs = run_completion(get_mistral_rubra_response, user_query)


Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"San Francisco","destination":"Cupertino","mode":"driving"}', name='calculate_distance')
Observation: {'role': 'tool', 'tool_call_id': '8eefe744', 'name': 'calculate_distance', 'content': 'Distance is 50 miles.'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What is the distance between San Francisco and Cupertino by driving and by air from both directions?'}, {'role': 'assistant', 'tool_calls': [{'id': '8eefe744', 'function': {'name': 'calculate_distance', 'arguments': '{"origin":"San Francisco","destination":"Cupertino","mode":"driving"}'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': '8eefe744', 'name': 'calculate_distance', 'content': 'Distance is 50 miles.'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"San Francisco","destination":"Cupertino","m

In [4]:
user_query0 = "create a 3d model with length 8, width 5, and height 4"
# user_query0 = "what's the latest insurance premium of 1, 3 ,5"
msgs = run_completion(get_mistral_rubra_response, user_query0)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"object_name":"rectangle","dimensions":{"length":8,"width":5,"height":4}}', name='create_3d_model')
Observation: {'role': 'assistant', 'tool_calls': [{'id': '0ee56562', 'function': {'name': 'create_3d_model', 'arguments': '{"object_name":"rectangle","dimensions":{"length":8,"width":5,"height":4}}'}, 'type': 'function'}]}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'create a 3d model with length 8, width 5, and height 4'}, {'role': 'assistant', 'tool_calls': [{'id': '0ee56562', 'function': {'name': 'create_3d_model', 'arguments': '{"object_name":"rectangle","dimensions":{"length":8,"width":5,"height":4}}'}, 'type': 'function'}]}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI response]:
  A 3D model of a rectangle with dimensions 8x5x4 has been successfully created.


In [5]:
user_query2 = "now order 3 umbrellas for me and generate a password of length 8"
msgs = run_completion(get_mistral_rubra_response, user_query2, msgs)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"number_to_buy":3}', name='orderUmbrella')
Observation: {'role': 'tool', 'tool_call_id': '19e16a34', 'name': 'orderUmbrella', 'content': 'Order placed. the price is 10 dollars.'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'now order 3 umbrellas for me and generate a password of length 8'}, {'role': 'assistant', 'tool_calls': [{'id': '19e16a34', 'function': {'name': 'orderUmbrella', 'arguments': '{"number_to_buy":3}'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': '19e16a34', 'name': 'orderUmbrella', 'content': 'Order placed. the price is 10 dollars.'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"length":8}', name='generate_password')
Observation: {'role': 'tool', 'tool_call_id': 'c14327a0', 'name': 'generate_password', 'content': 'Password generated: 85f87d3d'}
[{'ro

## Simple Math Chaining

In [6]:
user_query3 = "User tool to help me :  What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"


msgs = run_completion(get_mistral_rubra_response, user_query3)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"a":"4","b":"6"}', name='addition')
Observation: {'role': 'tool', 'name': 'addition', 'content': '10.0', 'tool_call_id': '89cdcf84'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'User tool to help me :  What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two'}, {'role': 'assistant', 'tool_calls': [{'id': '89cdcf84', 'function': {'name': 'addition', 'arguments': '{"a":"4","b":"6"}'}, 'type': 'function'}]}, {'role': 'tool', 'name': 'addition', 'content': '10.0', 'tool_call_id': '89cdcf84'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"a":"10.0","b":"5"}', name='multiplication')
Observation: {'role': 'tool', 'name': 'multiplication', 'content': '50.0', 'tool_call_id': '890a70de'}
[{'role': 'system', 'content': 'You are a hel

# Condition

In [7]:
user_query4 = "check the weather in boston, calculate the distance from boston to NYC for me only if it's less than 100 degrees Fahrenheit"
msgs = run_completion(get_mistral_rubra_response, user_query4)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: {'role': 'tool', 'tool_call_id': 'de8f0d7b', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "check the weather in boston, calculate the distance from boston to NYC for me only if it's less than 100 degrees Fahrenheit"}, {'role': 'assistant', 'tool_calls': [{'id': 'de8f0d7b', 'function': {'name': 'getCurrentWeather', 'arguments': '{"location":"Boston","unit":"f"}'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': 'de8f0d7b', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"Boston","destination":"New York City","mode":"car"}', name='calculate_distance')
Observation:

In [8]:
user_query5 = "check the weather in boston, calculate the distance from boston to NYC for me only if it's greater than 100 degrees Fahrenheit"
msgs = run_completion(get_mistral_rubra_response, user_query5)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: {'role': 'tool', 'tool_call_id': 'defe6bc9', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "check the weather in boston, calculate the distance from boston to NYC for me only if it's greater than 100 degrees Fahrenheit"}, {'role': 'assistant', 'tool_calls': [{'id': 'defe6bc9', 'function': {'name': 'getCurrentWeather', 'arguments': '{"location":"Boston","unit":"f"}'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': 'defe6bc9', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"Boston","destination":"New York City","mode":"car"}', name='calculate_distance')
Observati

# dependency

In [9]:
user_query6 = "check the size of all files in the 'documents' directory."
msgs = run_completion(get_mistral_rubra_response, user_query6)

Pointing to URL: http://localhost:8019/v1/

[AI calling functions]:
Tool Call: Function(arguments='{"directory":"documents"}', name='list_files')
Observation: {'role': 'tool', 'tool_call_id': 'e8b5f43a', 'name': 'list_files', 'content': 'File list:\nreport.docx\ntask.txt\nnotes.txt'}
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "check the size of all files in the 'documents' directory."}, {'role': 'assistant', 'tool_calls': [{'id': 'e8b5f43a', 'function': {'name': 'list_files', 'arguments': '{"directory":"documents"}'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': 'e8b5f43a', 'name': 'list_files', 'content': 'File list:\nreport.docx\ntask.txt\nnotes.txt'}]
Pointing to URL: http://localhost:8019/v1/
Loop 0

[AI response]:
  The files in the 'documents' directory are:
- report.docx
- task.txt
- notes.txt
